In [ ]:
sqlite3 .data/locations.db 

In [12]:
from sqlalchemy import create_engine, MetaData

# create an engine that connects to your database
engine = create_engine('sqlite+pysqlite:///.data/locations.db')


CREATE TABLE action (
    id INTEGER NOT NULL,     type VARCHAR NOT NULL,     encounter_id INTEGER NOT NULL, FOREIGN KEY (encounter_id) REFERENCES encounter(id),     critter_id INTEGER, FOREIGN KEY (critter_id) REFERENCES critter(id),     character_id INTEGER, FOREIGN KEY (character_id) REFERENCES character(id),     item_id INTEGER, FOREIGN KEY (item_id) REFERENCES item(id),     building_id INTEGER, FOREIGN KEY (building_id) REFERENCES building(id), PRIMARY KEY (id) );

CREATE TABLE building (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL,     enterable BOOLEAN NOT NULL,     location_id INTEGER, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

CREATE TABLE character (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL,     location_id INTEGER, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

CREATE TABLE critter (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,    

In [ ]:

# create a metadata object that will contain information about the database schema
metadata = MetaData()
metadata.reflect(bind=engine)


# define a helper function to generate a foreign key constraint for a column
def foreign_key_constraint(column, references_table, references_column='id'):
    return f"FOREIGN KEY ({column.name}) REFERENCES {references_table}({references_column})"

# iterate over the sorted list of tables in the database
for table in sorted(metadata.tables.values(), key=lambda t: t.name):
    # print the CREATE TABLE statement for this table
    print(f"CREATE TABLE {table.name} (")
    for column in table.columns:
        # print the column name and type
        print(f"    {column.name} {str(column.type).upper()}", end='')
        if not column.nullable:
            print(" NOT NULL", end='')
        print(",", end=' ')
        # add a foreign key constraint if applicable
        if column.foreign_keys:
            fk = next(iter(column.foreign_keys))
            print(foreign_key_constraint(column, fk.column.table.name, fk.column.name), end='')
            print(",", end=' ')
    # print the primary key constraint
    primary_keys = [key.name for key in table.primary_key]
    if primary_keys:
        print(f"PRIMARY KEY ({', '.join(primary_keys)})", end=' ')
    # close the CREATE TABLE statement
    print(");\n")


```sql
CREATE TABLE action (
    id INTEGER NOT NULL,     type VARCHAR NOT NULL,     encounter_id INTEGER NOT NULL, FOREIGN KEY (encounter_id) REFERENCES encounter(id),     critter_id INTEGER, FOREIGN KEY (critter_id) REFERENCES critter(id),     character_id INTEGER, FOREIGN KEY (character_id) REFERENCES character(id),     item_id INTEGER, FOREIGN KEY (item_id) REFERENCES item(id),     building_id INTEGER, FOREIGN KEY (building_id) REFERENCES building(id), PRIMARY KEY (id) );

CREATE TABLE building (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL,     enterable BOOLEAN NOT NULL,     location_id INTEGER, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

CREATE TABLE character (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL,     location_id INTEGER, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

CREATE TABLE critter (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL, PRIMARY KEY (id) );

CREATE TABLE encounter (
    id INTEGER NOT NULL,     probability FLOAT NOT NULL,     description VARCHAR NOT NULL,     location_id INTEGER NOT NULL, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

CREATE TABLE item (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL, PRIMARY KEY (id) );

CREATE TABLE location (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL, PRIMARY KEY (id) );

CREATE TABLE trigger (
    id INTEGER NOT NULL,     type VARCHAR NOT NULL,     way_id INTEGER, FOREIGN KEY (way_id) REFERENCES way(id),     building_id INTEGER, FOREIGN KEY (building_id) REFERENCES building(id),     encounter_id INTEGER NOT NULL, FOREIGN KEY (encounter_id) REFERENCES encounter(id), PRIMARY KEY (id) );

CREATE TABLE way (
    id INTEGER NOT NULL,     name VARCHAR NOT NULL,     description VARCHAR NOT NULL,     location_id INTEGER NOT NULL, FOREIGN KEY (location_id) REFERENCES location(id), PRIMARY KEY (id) );

```

```sql
sqlite> SELECT * FROM location WHERE id = 1;
1|Steampunk City of Atlantia|A bustling city of towering sky scrapers with clockwork gears and steam rising from the cobblestone streets. The streets are bustling with people and carriages and the air is thick with smoke and cogs. The city walls are tall and cold, with an entrance gate that only opens with a password
sqlite> SELECT * FROM character WHERE location_id = 1;
6|Goblin Mage|A powerful goblin mage with a staff, who will cast powerful spells on enemies|1
7|Eerie Figure|An eerie figure floating in the sky, it seems to be watching you.|1

```


Act as GameEngine. Characters

```
create GURPS character for character:
Goblin Mage|A powerful goblin mage with a staff, who will cast powerful spells on enemies
which is present at location:
Steampunk City of Atlantia|A bustling city of towering sky scrapers with clockwork gears and steam rising from the cobblestone streets. The streets are bustling with people and carriages and the air is thick with smoke and cogs. The city walls are tall and cold, with an entrance gate that only opens with a password

output it as INSERT INTO game_character (
    strength INTEGER,
    dexterity INTEGER,
    intelligence INTEGER,
    health INTEGER,
    will INTEGER,
    perception INTEGER,
    advantages TEXT,
    disadvantages TEXT,
    skills TEXT,
    character_id INTEGER REFERENCES character (id)
);


also output INSERT INTO game_item  (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT NOT NULL,
    weight REAL,
    value REAL,
    item_type TEXT,
    damage TEXT,
    armor INTEGER,
    range TEXT,
    durability INTEGER,
    rarity TEXT,
    enchantments TEXT[],
    character_id INTEGER REFERENCES character (id)
);



all approptiate items that this character has 

output all SQL in single code block with ``` for easy copying. strictly follow supplied schema of SQL tables, you only allowed to insert into game_item and game_character, record in table character already exists with id=6
```

```sql
INSERT INTO game_character (
    strength,
    dexterity,
    intelligence,
    health,
    will,
    perception,
    advantages,
    disadvantages,
    skills,
    character_id
)
VALUES (
    6, -- strength
    9, -- dexterity
    16, -- intelligence
    12, -- health
    14, -- will
    12, -- perception
    'Magical Aptitude: +4 to IQ when casting spells; Goblin Racial Skills: +2 to Stealth, Brawling, and Throwing', -- advantages
    'Bloodlust: must make a Will roll to avoid attacking wounded foes', -- disadvantages
    'Staff-15, Stealth-14, Throwing-14, Brawling-14, Innate Attack (Projectile)-16, Thaumatology-16', -- skills
    6 -- character_id
);

INSERT INTO game_item (
    id,
    name,
    description,
    weight,
    value,
    item_type,
    damage,
    armor,
    range,
    durability,
    rarity,
    enchantments,
    character_id
)
VALUES (
    1, -- id
    'Goblin Mage Staff', -- name
    'A staff made of darkwood that glows with an eerie green light. It seems to hum with magical energy.', -- description
    3.0, -- weight
    200.0, -- value
    'Weapon', -- item_type
    '2d-1 crushing, 1d-1 burning (staff)', -- damage
    0, -- armor
    'Thrown (12/24), Melee C', -- range
    20, -- durability
    'Rare', -- rarity
    '{Magical Aptitude +1}', -- enchantments
    6 -- character_id
);

INSERT INTO game_item (
    id,
    name,
    description,
    weight,
    value,
    item_type,
    damage,
    armor,
    range,
    durability,
    rarity,
    enchantments,
    character_id
)
VALUES (
    2, -- id
    'Goblin Mage Robe', -- name
    'A flowing robe made of black silk adorned with silver moons and stars. It crackles with magical energy.', -- description
    1.5, -- weight
    400.0, -- value
    'Armor', -- item_type
    '0', -- damage
    2, -- armor
    '0', -- range
    30, -- durability
    'Rare', -- rarity
    '{Magical Aptitude +1, Stealth +1}', -- enchantments
    6 -- character_id
);

INSERT INTO game_item (
    id,
    name,
    description,
    weight,
    value,
    item_type,
    damage,
    armor,
    range,
    durability,
    rarity,
    enchantments,
    character_id
)
VALUES (
    3, -- id
    'Goblin Mage Hat', -- name
    'A tall pointed hat made of black silk adorned with silver moons and stars. It crackles with magical energy.', -- description
    0.5, -- weight
    100.0, -- value
    'Armor', -- item_type
    '0', -- damage
    1, -- armor
    '0', -- range
    10, -- durability
    'Common', -- rarity
    '{Magical Aptitude +1}', -- enchantments
    6 -- character_id
);


```